In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
import datetime as dt

pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 400000)
pd.set_option("display.width", 1000)

sns.set_style("darkgrid")
sns.set_palette('muted')

plt.rcParams['figure.figsize'] = [24, 18]
plt.rcParams['figure.dpi'] = 150 # 200 e.g. is really fine, but slower

In [3]:
netflix = pd.read_excel('netflix_parsed.xlsx')
netflix = netflix[netflix['time'] < 200]
netflix.head()

,title,date,show_name,time
0,How I Met Your Mother: Season 7: The Magician'...,2021-01-24,How I Met Your Mother,21
1,How I Met Your Mother: Season 7: Good Crazy,2021-01-24,How I Met Your Mother,22
2,How I Met Your Mother: Season 7: Now We're Even,2021-01-24,How I Met Your Mother,22
3,How I Met Your Mother: Season 7: Trilogy Time,2021-01-24,How I Met Your Mother,21
4,How I Met Your Mother: Season 7: The Broath,2021-01-24,How I Met Your Mother,22


In [4]:
netflix_group = netflix.groupby(['show_name', 'date'])['time'].sum().reset_index()
netflix_group = netflix_group.pivot(index='date', columns='show_name', values='time').reset_index()
netflix_group = netflix_group.set_index('date')
netflix_group = netflix_group.fillna(0)

netflix_group = netflix_group.cumsum()


In [5]:
top_n = 2

top_names = list (netflix_group.loc['2021-01-24'].sort_values(ascending=False).index)[:top_n]

netflix_group = netflix_group[top_names]

In [6]:
netflix_group = netflix_group[netflix_group.sum().sort_values(ascending=True).index]

In [10]:
colors = plt.cm.tab20(range(100))
def init():
    ax.clear()
#     ax.set_xlim([0, netflix_group.drop(columns='date').values.max()])

def update(i):
    data = netflix_group
    y = data.columns
    width = data.iloc[i]
    date = data.index[i]
    ax.barh(y=y, width=width, tick_label=data.columns, color=colors)
    ax.set(title=f'Netflix Lockdown Viewing - {date}', xlabel='Minutes')
    
    rects = ax.patches

    # For each bar: Place a label
    for rect in rects:
        # Get X and Y placement of label from rect.
        x_value = rect.get_width()
        y_value = rect.get_y() + rect.get_height() / 2

        # Number of points between bar and label. Change to your liking.
        space = 5
        # Vertical alignment for positive values
        ha = 'left'

        # If value of bar is negative: Place label left of bar
        if x_value < 0:
            # Invert space to place label to the left
            space *= -1
            # Horizontally align label at right
            ha = 'right'

        # Use X value as label and format number with one decimal place
        label = "{:.1f}".format(x_value)

        # Create annotation
        ax.annotate(
            label,                      # Use `label` as label
            (x_value, y_value),         # Place label at end of the bar
            xytext=(space, 0),          # Horizontally shift label by `space`
            textcoords="offset points", # Interpret `xytext` as offset in points
            va='center',                # Vertically center label
            ha=ha)                      # Horizontally align label differently for
                                        # positive and negative values.
    
    
fig = plt.Figure(figsize=(12, 8), dpi=200, tight_layout=True)
ax = fig.add_subplot()
anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(netflix_group), 
                     interval=100, repeat=False)

In [ ]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)


In [ ]:
anim.save('netflixbinge.mp4')